In [1]:
#%pip install equinox openmdao[all]

In [2]:
import equinox as eqx
from util import generate_base_model, density, dyn_viscousity
import jax
import jax.numpy as jnp

In [3]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"models/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"models/drag_surrogate.eqx", base_model)

In [4]:
from optimizer import Optimizer

In [5]:
opt = Optimizer()

In [6]:
opt.load_surrogates(lift_model, drag_model)

In [7]:
opt.define_material(
    E=69e9,
    rho=2700,
    yield_strength=241e6,
    shear_strength=207e6
)

In [8]:
altitude = 6700

results = opt.solve_wing(
    lift_goal=18780,
    safety_factor=0.75,
    v_infty=110,
    rho=density(altitude),
    mu=dyn_viscousity(altitude),
    alpha_geo=jnp.deg2rad(jnp.float32(1))
)

Optimizing planform
Optimization terminated successfully    (Exit mode 0)
            Current function value: 530.250454965089
            Iterations: 4
            Function evaluations: 6
            Gradient evaluations: 4
Optimization Complete
-----------------------------------
Optimizing airfoil
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0034212841171024876
            Iterations: 31
            Function evaluations: 45
            Gradient evaluations: 31
Optimization Complete
-----------------------------------
Optimizing spars
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.02319789080124113
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------


In [9]:
from util import pretty_print

pretty_print(results)

Airfoil
----------
Base Shape Coef. 1.687
Max Thickness    0.185
Taper Exponent   2.833
Max Camber       0.000
Camber Exponent  0.953
Reflex           -0.007

Planform
----------
Wingspan         14.353 m
Chord length     1.000 m
Aspect Ratio     14.353

Spars
----------
Flange Height    0.005 m
Flange Width     0.100 m
Web Height       0.117 m
Web Width        0.005 m
Spar Ratio       7.39
Main Spar Pos.   0.05c
Rear Spar Pos.   0.95c

Aerodynamics & Structure
----------
Lift             18734
Drag             343
Cl at alpha=0    0.32
Alpha Cl=0       -2.95 deg
Max τ threshold  0.36%
Max σ threshold  21.90%
Material Usage   0.023 m3

Timing
----------
Planform         15 seconds
Airfoil          4 seconds
Spars            3 seconds
Total            23 seconds
